In [6]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw
import numpy as np
import io
import gradio as gr

# Authenticate client
prediction_key = "07051a00c54f429aacf110c380a4e3fd"
prediction_endpoint = "https://team1thetteamcv-prediction.cognitiveservices.azure.com/"
project_id = "240ecda2-da48-4e42-8e67-107e60f73f3d"
model_name = "Iteration2"

credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(endpoint=prediction_endpoint, credentials=credentials)

data = {
    "Inputs": {
        "input1": [
            {
                "paper": "이건 종이야!📄\n종이는 종이끼리 모여진 데 버려주면 돼!",
                "glass": "이건 유리야!🔍\n유리병은 깨질 수 있으니 던지지 말고 조심히 버려줘!",
                "metal": "이건 금속이야!🔧\n날카로운 모서리에 손이 베이지 않게 조심해!",
                "plastic": "이건 플라스틱이야!🌱\n페트병은 라벨을 떼고 찌그러트려 버려줘!",
                "trash": "이건 일반쓰레기야!🗑️\n일반쓰레기는 종량제 봉투에 모아줄래?\n종량제 봉투가 뭐냐구? 함께 알아볼까?"
            }
        ]
    },
    "GlobalParameters": {}
}

def process_image(image):
    # 이미지 리사이즈
    max_size = (512, 512)
    image.thumbnail(max_size)
    
    # JPEG로 변환
    byte_arr = io.BytesIO()
    image.save(byte_arr, format='JPEG')
    
    return byte_arr.getvalue()

def predict_image(input_img):
    processed_image = process_image(input_img)
    h, w = input_img.size
    
    results = predictor.classify_image(project_id, model_name, processed_image)  # 이미지 분류용 메서드
    
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.axis('off')
    
    draw = ImageDraw.Draw(input_img)
    lineWidth = int(w / 100)
    color = 'magenta'
    
    detected_objects = []
    
    for prediction in results.predictions:
        if (prediction.probability * 100) > 60:
            detected_objects.append(prediction.tag_name)
            draw.text((10, 10 + len(detected_objects) * 20), f"{prediction.tag_name}: {prediction.probability * 100:.2f}%", fill=color)

        # bounding_box의 경우 classify_image(분류) 메서드를 사용하는 게 아닌, detect_image(객체 탐지) 메서드를 사용 중인 경우에 가능하다.
        # 현재 진행 중인 프로젝트의 경우 classify_image 메서드를 사용하여 분류 작업을 하는 중이기에, 사용이 불가능하다.
        # 만약 detect_image 메서드를 새롭게 만든 후 여러가지 이미지를 다시 학습 시키면 되겠지만.. 너무 노가다인데..?
    
    ax.imshow(input_img)
    
    # 예측 결과 텍스트 생성
    if detected_objects:
        predictions_text = "\n".join(data["Inputs"]["input1"][0].get(obj, "무슨 쓰레기인지 잘 모르겠어. 주위를 정리하고 사진을 다시 찍어줄래?") for obj in detected_objects)
    else:
        predictions_text = "무슨 쓰레기인지 잘 모르겠어. 주위를 정리하고 사진을 다시 찍어줄래?"
    
    return fig, predictions_text

result_output = gr.Textbox(label="Result")

demo = gr.Interface(
    fn=predict_image, 
    inputs=gr.Image(type='pil'), 
    outputs=[gr.Plot(), result_output]
)
demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7910

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
